
# Advisory mission for the Boston municipal authorities


![title](assets/boston_logo.png)

Welcome to the Decision Science exercise of your data certification exam!

Here are a few words to describe how the `decision_science` directory is organised:

In [1]:
# Start by running this cell to see the architecture of the directory
!tree

.
├── boston_crimes.ipynb
├── data
├── data.py
├── db
├── README.md
└── tests

3 directories, 3 files


- the `boston_crimes.ipynb` notebook that you currently have under your eyes is the main document. You will find all the instructions here and except when it is explicitly specified, you should provide all your answers in this notebook;


- the `data` and `db` folders will be filled-in throughout the exercise respectively with `.csv` datasets and a `.sqlite` file, for you to run all your analyses; 


- you will not have to interact with the `assets` folder for this exercise;


- the `tests` folder will contain all the `.pickle` files that will be saved throughout the exercise with your key findings. Please run all the "Save your results" cells when completing the exercise!

⚠️ **Important remark** before you dive into the exercise. This notebook is quite long and it is easy to get lost in it: take full advantage of the collapsible headers and of the table of content. If you have not yet activated these Jupyter Notebook extensions, you may consider doing so now!

# Imports

You can use this section to run your imports in a centralised manner throughout the exercise.

In [14]:
# Load the nbresult package to be able to save your results 
from nbresult import ChallengeResult

In [15]:
# Useful import for data collection
import sqlite3

In [16]:
# Useful imports for data manipulation and analysis
import numpy as np
import pandas as pd

In [17]:
# Useful imports for data visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [18]:
# Useful imports to estimate regression models
import statsmodels.formula.api as smf

# 1. Analysis for the mayor's team

During the last municipal campaign in Boston, criminality has been a major topic of debates. As citizens have expressed strong expectations from her on that front, the newly-elected mayor of Boston is looking for data-based insights on criminality in the Massachussetts capital. She has mandated your economics and urbanism consulting firm, *The Locomotive*, for this study.

## 1.1 Load the database

Download the `boston_crimes.sqlite` database from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite) and store it inside the `db` folder.

In [11]:
# You may directly run this cell to do so
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/boston_crimes.sqlite > db/boston_crimes.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.1M  100 19.1M    0     0  1032k      0  0:00:18  0:00:18 --:--:-- 1073k


## 1.2 Familiarize yourself with the database

This section aims at giving you a first overview of the database. 

As you will see, it consists in three tables: 

- the `incident_reports` table has been provided by the Police Department of Boston. Each observation corresponds to a criminal incident that has required an intervention by the police in the municipality of Boston;  



- the `districts` table has been provided by the Urbanism Department of Boston. It gathers geographical information about the various police districts of Boston;  



- and the `indicators` table has been shared by the Economics Department of Boston, which keeps track of various indicators of the social and economic activity of Boston neighborhoods. Each observation corresponds to a police district.

More information about the different fields of these three tables can be found in the dedicated `README.md` file

### Connect to the database

**🔍 Using your preferred SQL client, connect to the database and browse through it as you wish to get acquainted with the data.**

### Draw a schema of the database

**📝 Draw the database schema thanks to the [schema editor](https://kitt.lewagon.com/db) on Kitt.**

**📝 Download the schema and save it as `boston_crimes.xml` in the `db` folder.**

## 1.3 Extract the relevant dataset

Now that you have a good overview of the database, you can kick off the work! You will start with an SQL query to gather the relevant information.

### Build the dataset

We want to investigate the influence of the socio-economic characteristics of Boston's different districts on the number of crime reports and incidents. To do so, we need to extract the relevant dataset. **Each row should correspond to one of the 12 police districts of Boston** (as listed in the `districts` table of the database).

To identify the district, we will need **the following columns**: 

- the `CODE` of the police district (1 letter and 1 or 2 numbers);
- the full `NAME` of the police district.

Additionally, you will need to **create an additional field** (which will serve as dependent variable in future regressions): `NB_INCIDENTS`, i.e. the total number of incidents reported in the police district over the period covered by the data at hand (2015-2019).

Eventually, we want the dataset to **include several socio-economic indicators**:

- `MEDIAN_AGE`;
- `TOTAL_POP`;
- `PERC_OF_30_34`;
- `PERC_MARRIED_COUPLE_FAMILY`;
- `PER_CAPITA_INCOME`;
- `PERC_OTHER_STATE_OR_ABROAD`;
- `PERC_LESS_THAN_HIGH_SCHOOL`;
- `PERC_COLLEGE_GRADUATES`. 

Overall, your dataset should comprise 12 rows and 11 columns.

Eventually, note that **the resulting DataFrame must be ordered by the number of incidents**, from the largest to the smallest total.

**📝 Write the SQL query you need to fetch the data. Save it as a `str` in the `query` variable.**

In [12]:
import sqlite3

conn = sqlite3.connect("db/boston_crimes.sqlite")
c = conn.cursor()

In [22]:
query = """
    SELECT
        districts.CODE,
        districts.NAME,
        COUNT(*) NB_INCIDENT,
        indicators.MEDIAN_AGE,
        indicators.TOTAL_POP,
        indicators.PERC_OF_30_34,
        indicators.PERC_MARRIED_COUPLE_FAMILY,
        indicators.PER_CAPITA_INCOME,
        indicators.PERC_OTHER_STATE_OR_ABROAD,
        indicators.PERC_LESS_THAN_HIGH_SCHOOL,
        indicators.PERC_COLLEGE_GRADUATES
    FROM districts
    JOIN incident_reports ON districts.CODE = incident_reports.DISTRICT
    JOIN indicators ON districts.CODE = indicators.CODE
    GROUP BY incident_reports.DISTRICT
    ORDER BY NB_INCIDENT DESC
"""

**📝 Store the output of the query in a `DataFrame` named `crimes_df`. Display the 5 first rows, as well as the shape of the dataset.**

In [26]:
crimes_df = pd.read_sql_query(query, conn)

print('Shape of crimes_df : {}'.format(crimes_df.shape))

print()
print('###')
print('crimes_df 5 first rows below')
print()

crimes_df.head()

Shape of crimes_df : (12, 11)

###
crimes_df 5 first rows below



CODE        NAME  NB_INCIDENT  MEDIAN_AGE  TOTAL_POP  PERC_OF_30_34  \
0   B2     Roxbury        38877        32.5      54161           27.8   
1  C11  Dorchester        32875        33.4     126909           28.2   
2   D4   South End        31258        37.1      32571           33.9   
3   B3    Mattapan        28331        36.7      26659           20.9   
4   A1    Downtown        26260        33.5      18306           32.5   

   PERC_MARRIED_COUPLE_FAMILY  PER_CAPITA_INCOME  PERC_OTHER_STATE_OR_ABROAD  \
0                        17.8              20978                         2.9   
1                        26.6              29767                         2.4   
2                        28.3              83609                         6.2   
3                        29.8              28356                         2.3   
4                        35.8              80057                        14.8   

   PERC_LESS_THAN_HIGH_SCHOOL  PERC_COLLEGE_GRADUATES  
0                        23.0                    18.9  
1                        18.0                    17.1  
2                        11.8                     8.5  
3                        14.5                    22.9  
4                        15.4                     6.9

### Save your results

You can run the following cell to save your results:

In [27]:
ChallengeResult('sql', query=query, data=crimes_df).write()

## 1.4 Linear regression - The socio-economic determinants of criminality

As mentioned above, we want to investigate the impact of the socio-economic characteristics of the different Boston police districts on the number of incidents that are reported in these areas. 
- We are going to use the number of incidents as dependent variable 
- our regressors will be the various socio-economic indicators extracted from the database.

### 1.4.1 Start from a fresh dataset

To make sure that you are using the right data, you can load a fresh dataset from this [URL](https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/regression.csv).

**📝 Load the data into a DataFrame named `data`**

In [28]:
data = pd.read_csv('boston_crimes_regression.csv')

print('Shape of data : {}'.format(data.shape))

print()
print('###')
print('data 5 first rows below')
print()

data.head()

Shape of data : (12, 11)

###
data 5 first rows below



MEDIAN_AGE  TOTAL_POP  PERC_OF_30_34  PERC_MARRIED_COUPLE_FAMILY  \
0        30.8      55297           52.8                        26.4   
1        35.7      19890           28.2                        36.4   
2        33.4     126909           28.2                        26.6   
3        33.5      18306           32.5                        35.8   
4        30.6      47263           31.1                        30.4   

   PER_CAPITA_INCOME  PERC_OTHER_STATE_OR_ABROAD  PERC_LESS_THAN_HIGH_SCHOOL  \
0              41261                         8.6                         6.7   
1              75339                         3.4                         7.9   
2              29767                         2.4                        18.0   
3              80057                        14.8                        15.4   
4              31473                         3.5                        27.2   

   PERC_COLLEGE_GRADUATES CODE  NB_INCIDENTS         NAME  
0                    10.5  D14         13788     Brighton  
1                     8.2  A15          4765  Charlestown  
2                    17.1  C11         32875   Dorchester  
3                     6.9   A1         26260     Downtown  
4                    11.5   A7          9691  East Boston

In [38]:
data.isnull().sum().sort_values(ascending=False)/len(data)

NAME                          0.0
NB_INCIDENTS                  0.0
CODE                          0.0
PERC_COLLEGE_GRADUATES        0.0
PERC_LESS_THAN_HIGH_SCHOOL    0.0
PERC_OTHER_STATE_OR_ABROAD    0.0
PER_CAPITA_INCOME             0.0
PERC_MARRIED_COUPLE_FAMILY    0.0
PERC_OF_30_34                 0.0
TOTAL_POP                     0.0
MEDIAN_AGE                    0.0
dtype: float64

> There is no missing value

In [32]:
data.dtypes

MEDIAN_AGE                    float64
TOTAL_POP                       int64
PERC_OF_30_34                 float64
PERC_MARRIED_COUPLE_FAMILY    float64
PER_CAPITA_INCOME               int64
PERC_OTHER_STATE_OR_ABROAD    float64
PERC_LESS_THAN_HIGH_SCHOOL    float64
PERC_COLLEGE_GRADUATES        float64
CODE                           object
NB_INCIDENTS                    int64
NAME                           object
dtype: object

> All the features are numerical. I won't need to use object preprocessing

In [29]:
data.describe()

MEDIAN_AGE      TOTAL_POP  PERC_OF_30_34  PERC_MARRIED_COUPLE_FAMILY  \
count   12.000000      12.000000      12.000000                   12.000000   
mean    34.933333   44262.083333      31.100000                   31.208333   
std      3.638764   28579.919064      10.038109                    7.436453   
min     30.600000   18306.000000      18.100000                   17.800000   
25%     32.350000   31093.000000      26.125000                   26.550000   
50%     34.150000   37848.000000      29.650000                   30.100000   
75%     36.800000   48987.500000      32.850000                   35.950000   
max     42.800000  126909.000000      52.800000                   46.200000   

       PER_CAPITA_INCOME  PERC_OTHER_STATE_OR_ABROAD  \
count          12.000000                   12.000000   
mean        48985.000000                    4.650000   
std         22002.128393                    3.795572   
min         20978.000000                    1.900000   
25%         31046.500000                    2.375000   
50%         44548.500000                    3.150000   
75%         67393.500000                    5.675000   
max         83609.000000                   14.800000   

       PERC_LESS_THAN_HIGH_SCHOOL  PERC_COLLEGE_GRADUATES  NB_INCIDENTS  
count                   12.000000               12.000000     12.000000  
mean                    13.391667               13.291667  19768.416667  
std                      6.700401                5.464839  11138.568902  
min                      6.500000                6.900000   4765.000000  
25%                      7.900000                8.475000  11836.000000  
50%                     12.800000               11.800000  15202.500000  
75%                     16.050000               17.550000  29062.750000  
max                     27.200000               22.900000  38877.000000

> The features need to be standardized

### 1.4.2 Run the regression and output its summary

Thanks to the Statsmodels Formula API, we will run the regression described below. 

The dependent variable (or target variable) should be **the total number of incidents** reported in each police district.

We will focus on the following regressors: 

- the **median age** in the district, whose effect is difficult to anticipate on the number of crimes;
 
 
- the **percentage of 30-34 years old** in the district, whose effect is also unclear a priori;
 
 
- the **share of families with a married couple** among all households, which could be anticipated to have a negative effect on criminality (more attention to safety among residents...);
 
 
- the **percentage of residents having moved from abroad or from another US state over the last year**, mobility being often associated with social marginalisation and possibly with a higher risk of resorting to illegal activities;
 
 
- the **percentage of residents having stopped their studies before getting a high school degree**. Economic models would suggest that due to the more narrow job opportunities to which this group has access, the incentive is stronger to resort to illicit activities;
 
 
- the **percentage of college graduates** in the district, which we would expect to have an opposite effect.
 
**📝 Based on these indications, estimate the linear regression model and output its summary in this section of the notebook. Store the estimated model inside a `model` variable.**

---
#### Features exploration

In [33]:
data.columns

Index(['MEDIAN_AGE', 'TOTAL_POP', 'PERC_OF_30_34',
       'PERC_MARRIED_COUPLE_FAMILY', 'PER_CAPITA_INCOME',
       'PERC_OTHER_STATE_OR_ABROAD', 'PERC_LESS_THAN_HIGH_SCHOOL',
       'PERC_COLLEGE_GRADUATES', 'CODE', 'NB_INCIDENTS', 'NAME'],
      dtype='object')

In [34]:
model_features = ['MEDIAN_AGE', 'PERC_OF_30_34', 'PERC_MARRIED_COUPLE_FAMILY', 'PERC_OTHER_STATE_OR_ABROAD', 'PERC_LESS_THAN_HIGH_SCHOOL', 'PERC_COLLEGE_GRADUATES']

X = data[model_features]

y = data['NB_INCIDENTS']

In [37]:
X.head()

MEDIAN_AGE  PERC_OF_30_34  PERC_MARRIED_COUPLE_FAMILY  \
0        30.8           52.8                        26.4   
1        35.7           28.2                        36.4   
2        33.4           28.2                        26.6   
3        33.5           32.5                        35.8   
4        30.6           31.1                        30.4   

   PERC_OTHER_STATE_OR_ABROAD  PERC_LESS_THAN_HIGH_SCHOOL  \
0                         8.6                         6.7   
1                         3.4                         7.9   
2                         2.4                        18.0   
3                        14.8                        15.4   
4                         3.5                        27.2   

   PERC_COLLEGE_GRADUATES  
0                    10.5  
1                     8.2  
2                    17.1  
3                     6.9  
4                    11.5

---
#### Linear Regression

In [60]:
model = smf.ols(formula='NB_INCIDENTS ~ MEDIAN_AGE + PERC_OF_30_34 + PERC_MARRIED_COUPLE_FAMILY + PERC_OTHER_STATE_OR_ABROAD + PERC_LESS_THAN_HIGH_SCHOOL + PERC_COLLEGE_GRADUATES', data=data)

In [61]:
model = model.fit()

In [62]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:           NB_INCIDENTS   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.777
Method:                 Least Squares   F-statistic:                     7.391
Date:                Tue, 29 Jun 2021   Prob (F-statistic):             0.0222
Time:                        11:48:58   Log-Likelihood:                -114.59
No. Observations:                  12   AIC:                             243.2
Df Residuals:                       5   BIC:                             246.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept           

/home/gaetan/.pyenv/versions/3.8.5/envs/lewagon/lib/python3.8/site-packages/scipy/stats/stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=12
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


### Save your results

You can run the following cell to save your results:

In [65]:
ChallengeResult(
    'linear_regression_model',
    data=data,
    model=model
).write()

### 1.4.3 Statistical inference questions 🤯

You will now be asked a series of statistical inference and methodological questions about the regression model estimated above. Don't worry if results do not perfectly fit the "predictions" that we made earlier about the impact of each regressor: the goal was to form an *a priori* hypothesis, which is validated or invalidated through this empirical analysis.

#### Questions on the results

**❓ Is the median age associated with a positive (increasing the number of crimes) or a negative (decreasing the target variable) effect on the number of crime incidents? Simply write your answer as a string below**

In [67]:
answer_median_age = '''
The median age is associated with a positive effect on the number of crime indidents
'''

**❓ What is the t-statistic associated with the median age regressor? How is it computed?**

💡 Hint: We are looking at a ratio

In [68]:
answer_t_statistic = '''
It is a measure of the median age regressor's precision.

t = MEDIAN_AGE_coef / MEDIAN_AGE_std_err
'''

**📝 Recompute approximately the t-statistic based on the regression summary.**

As it is a ratio $t = \frac{numerator}{denominator}$:
- Store the numerator into a `numerator` variable
- Store the denominator into a `denominator` variable
- Store the t-statistic into a `t_median_age` variable

In [72]:
summary = pd.read_html(result.summary().tables[1].as_html(),header=0,index_col=0)[0]

In [76]:
summary

coef    std err      t  P>|t|      [0.025  \
Intercept                   13580.0000  61600.000  0.220  0.834 -145000.000   
MEDIAN_AGE                   2252.7344   1237.522  1.820  0.128    -928.416   
PERC_OF_30_34                -532.9973    468.940 -1.137  0.307   -1738.447   
PERC_MARRIED_COUPLE_FAMILY  -2115.1839    408.344 -5.180  0.004   -3164.866   
PERC_OTHER_STATE_OR_ABROAD   1787.6939    532.432  3.358  0.020     419.034   
PERC_LESS_THAN_HIGH_SCHOOL    112.7375    493.977  0.228  0.829   -1157.072   
PERC_COLLEGE_GRADUATES         19.4699    458.775  0.042  0.968   -1159.850   

                                0.975]  
Intercept                   172000.000  
MEDIAN_AGE                    5433.885  
PERC_OF_30_34                  672.452  
PERC_MARRIED_COUPLE_FAMILY   -1065.502  
PERC_OTHER_STATE_OR_ABROAD    3156.354  
PERC_LESS_THAN_HIGH_SCHOOL    1382.547  
PERC_COLLEGE_GRADUATES        1198.790

In [78]:
numerator = summary['coef'].values[1]
denominator = summary['std err'].values[1]
t_median_age = numerator / denominator

In [79]:
t_median_age

1.8203590724043692

**❓ What is the p-value associated with the median age regressor? Store it in the `pvalue_median_age` variable (you may directly copy-paste it from the regression summary).** 

In [81]:
pvalue_median_age = summary['P>|t|'].values[1]

pvalue_median_age

0.128

**❓ What does this p-value mean for the median age? Is its effect statistically significant at the 95% confidence level? At the 90% confidence level? Simply formulate your answer in the cell below.**

In [91]:
answer_p_value = """
The p-value for median age tests the null hypothesis that this feature has no effect on our model

A p-value at 0.128 is not statistically significant at 95% or 90% confidence level
"""

**❓ What are the two regressors whose effect is statistically significant at the 95% confidence level in this regression model? Store the name of the variables as a list of strings in the `significant_regressors` variable.**

In [83]:
significant_regressors = ['PERC_MARRIED_COUPLE_FAMILY', 'PERC_OTHER_STATE_OR_ABROAD']

print('Variables statistically significant at the 95% confidence level in our model : {} and {}'.format(significant_regressors[0], significant_regressors[1]))

Variables statistically significant at the 95% confidence level in our model : PERC_MARRIED_COUPLE_FAMILY and PERC_OTHER_STATE_OR_ABROAD


**❓ Holding all other regressors constant, by how much does the total number of incidents increase or decrease when the share of families with a married couple increases by 1 percentage point in the district? Please formulate a full sentence with statistical rigor!**

In [88]:
answer_how_much_increase = """
The number of incidents increases by 11464,8161 with a standard deviation of 408.344 (between 11056.4721 and 11873.1601)

nb_incidents_predicted = prediction = 1 * -2115.1839 + 13580.0000 

interval = prediction - 408.344, prediction + 408.344
"""

#### Limits of this regression model

You had asked the intern on the team to estimate a linear regression model so as to investigate the socio-economic determinants of crime in Boston. The results above are those that he presented. In the email he sent to you, he added:

> *You will probably notice the extremely high R-squared score of this model: I think we have an excellent fit and the results are solid* 😄

But you have very strong doubts about this regression and you think it is a perfect occasion to give some very important advice to your intern...

**❓  What is the main limitation of this (clearly spurious) regression according to you? This observation explains why we are getting a very high R-squared and large standard errors. Please provide your answer in the following Markdown cell.**

In [89]:
answer_limitations = """
The variables weren't scalled which explain the big coefficients. Furthermore, we have only 12 observations and didn't use the holdout method to be sure that our model is not overfitted.

Even if it is difficult to use a train/test split here with the number of observations
"""

### Save your results

You can run the following cell to save your results:

In [92]:
import json
answers_inference = {"MEDIAN_AGE": answer_median_age,
                    "T_STAT":answer_t_statistic,
                     "P_VALUE": answer_p_value,
                     "INCREASE": answer_how_much_increase,
                     "LIMITATIONS": answer_limitations}

with open("tests/answers_inference.json", "w", encoding="utf-8") as f:
    json.dump(answers_inference, f, ensure_ascii=False, indent=4)

ChallengeResult(
    'linear_regression_analysis',
    model=model,
    numerator=numerator,
    denominator=denominator,
    t=t_median_age,
    pvalue=pvalue_median_age,
    regressors=significant_regressors
).write()

# 2. Analysis for the police department

The head of the Police Department of Boston, who read your report for the Mayor's team, was extremely interested in the results. He contacted your consulting firm for an additional presentation, that would focus on the nature of crimes that take place in Boston, the potential time trends that you could identify and/or the heterogeneity of the impact of criminality on the different police districts. 

## 2.1 Start with a fresh dataset

You will start from a fresh dataset, that corresponds more or less to the `incident_reports` table of the database.

In [93]:
# Run this cell to download the datasets in the data directory
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/incident_reports.csv > data/incident_reports.csv   
!curl https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/districts.csv > data/districts.csv 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 19.1M  100 19.1M    0     0   581k      0  0:00:33  0:00:33 --:--:--  377k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   498  100   498    0     0   1257      0 --:--:-- --:--:-- --:--:--  1254


In [1]:
# Load the dataset
from data import load_data_viz_data

data = load_data_viz_data()
data[['LAT','LONG']] = data[['LAT','LONG']].astype('float64')

print("Shape of the DataFrame:", data.shape)

data.head()

Shape of the DataFrame: (237221, 9)


INCIDENT_NUMBER       OFFENSE_CODE_GROUP  SHOOTING     OCCURRED_ON_DATE  \
0      I192068249                    Other         0  2015-08-28 10:20:00   
1      I182074094  Violence and harassment         0  2015-09-14 09:31:00   
2      I182054888  Violence and harassment         0  2015-07-12 15:37:00   
3      I182054888                    Other         0  2015-07-12 15:37:00   
4      I182054888                    Other         0  2015-07-12 15:37:00   

         LAT       LONG     NAME  LAT_POLICE_STATION  LONG_POLICE_STATION  
0  42.330119 -71.084251  Roxbury           42.328894           -71.085359  
1  42.315142 -71.067047  Roxbury           42.328894           -71.085359  
2  42.312243 -71.075499  Roxbury           42.328894           -71.085359  
3  42.312243 -71.075499  Roxbury           42.328894           -71.085359  
4  42.312243 -71.075499  Roxbury           42.328894           -71.085359

## 2.1 Further data manipulations

In this section, we are going to answer two specific questions that the head of the Police Department of Boston asked your team, about the characteristics of the incidents that occur in the various districts of the municipality.

### 2.1.1 Most represented type of incident per district

First, the head of the Police Department of Boston wants to know what incident category is most often found in each district. 

**📝 Construct a DataFrame with**
- **one row per police district, designated by its full name**
- **one text column that indicates the name of the most common category of incident in the district over the whole sample period.**

💡 Hint: you may need to first define a custom aggregation function.

In [7]:
list(data.NAME.unique())

['Roxbury',
 'Dorchester',
 'Downtown',
 'Hyde Park',
 'South End',
 'Mattapan',
 'Brighton',
 'East Boston',
 'West Roxbury',
 'South Boston',
 'Jamaica Plain',
 'Charlestown']

In [10]:
data[data.NAME == 'South Boston']['OFFENSE_CODE_GROUP'].value_counts()

Larceny and vandalism             5637
Police investigation procedure    3523
Violence and harassment           2013
Drugs and disorderly conduct      1867
Fraud and law violations          1654
Other                             1327
Disputes                           596
Name: OFFENSE_CODE_GROUP, dtype: int64

In [45]:
def find_code_group(classment):
    df_MRA_dict = {}

    for district in list(data.NAME.unique()):
        df_MRA_dict[district] = data[data.NAME == district]['OFFENSE_CODE_GROUP'].value_counts().index[classment - 1]

    df_MRA = pd.DataFrame.from_dict(df_MRA_dict, orient='index')

    return df_MRA

In [71]:
find_code_group(3)

0
Roxbury         Violence and harassment
Dorchester      Violence and harassment
Downtown        Violence and harassment
Hyde Park       Violence and harassment
South End       Violence and harassment
Mattapan        Violence and harassment
Brighton       Fraud and law violations
East Boston     Violence and harassment
West Roxbury   Fraud and law violations
South Boston    Violence and harassment
Jamaica Plain   Violence and harassment
Charlestown     Violence and harassment

**❓ Can you tell what is the second most common offense int the Brighton district?**

In [65]:
find_code_group(2)

0
Roxbury        Police investigation procedure
Dorchester     Police investigation procedure
Downtown       Police investigation procedure
Hyde Park      Police investigation procedure
South End      Police investigation procedure
Mattapan                Larceny and vandalism
Brighton       Police investigation procedure
East Boston    Police investigation procedure
West Roxbury   Police investigation procedure
South Boston   Police investigation procedure
Jamaica Plain  Police investigation procedure
Charlestown    Police investigation procedure

In [66]:
second_most_common_offence = "Police investigation procedure"

### Average distance to the police station per district

Second, based on the Haversine distance function defined below, the head of the Police Department would like to know, for each district, **the average distance between the location of the incident and the police station**. 

**📝 Construct a DataFrame with one row per police district, designated by its full name, and one column that displays this average Haversine distance.**

In [50]:
# Haversine distance function
from math import radians, sin, cos, asin, sqrt

def haversine_distance(lon1, lat1, lon2, lat2):
    """
    Compute distance (km) between two pairs of (lat, lng) coordinates
    See - (https://en.wikipedia.org/wiki/Haversine_formula)
    """
    
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    
    return 2 * 6371 * asin(sqrt(a))

In [51]:
data.head()

INCIDENT_NUMBER       OFFENSE_CODE_GROUP  SHOOTING     OCCURRED_ON_DATE  \
0      I192068249                    Other         0  2015-08-28 10:20:00   
1      I182074094  Violence and harassment         0  2015-09-14 09:31:00   
2      I182054888  Violence and harassment         0  2015-07-12 15:37:00   
3      I182054888                    Other         0  2015-07-12 15:37:00   
4      I182054888                    Other         0  2015-07-12 15:37:00   

         LAT       LONG     NAME  LAT_POLICE_STATION  LONG_POLICE_STATION  
0  42.330119 -71.084251  Roxbury           42.328894           -71.085359  
1  42.315142 -71.067047  Roxbury           42.328894           -71.085359  
2  42.312243 -71.075499  Roxbury           42.328894           -71.085359  
3  42.312243 -71.075499  Roxbury           42.328894           -71.085359  
4  42.312243 -71.075499  Roxbury           42.328894           -71.085359

In [54]:
data['H_DISTANCE'] = data.apply(lambda row: haversine_distance(row['LONG'], row['LAT'], row['LONG_POLICE_STATION'], row['LAT_POLICE_STATION']),axis=1)

**❓ Can you tell what is the average distance between the police station and the offenses in the Brighton district?**

In [76]:
df_HDIS_dict = {}

for district in list(data.NAME.unique()):
    df_HDIS_dict[district] = data[data.NAME == district]['H_DISTANCE'].mean()

df_HDIS = pd.DataFrame.from_dict(df_HDIS_dict, orient='index')

In [77]:
df_HDIS

0
Roxbury         2.989029
Dorchester      3.174444
Downtown        6.336657
Hyde Park       2.397613
South End       1.862039
Mattapan        1.908636
Brighton        3.770867
East Boston     5.568626
West Roxbury   18.370045
South Boston   31.454220
Jamaica Plain   2.585737
Charlestown    58.758394

In [61]:
average_distance_km = df_HDIS.loc['Brighton']

In [67]:
result = ChallengeResult('manipulation',
                         second_most_common_offence=second_most_common_offence,
                         average_distance_km=average_distance_km)
result.write()

# 3. Short presentation (REQUIRED TO VALIDATE THE CERTIFICATION)
🚨🚨🚨🚨🚨🚨

Using the latest dataset that you loaded, your mission is now to prepare 5 slides (including a title slide) that you would present to the head of the Police Department. You may or may not, as you prefer, include the output of the two "Further data manipulations" tasks in your presentation.

⚠️  You can use any presentation editor of your choice, but **the slides must be shared either in HTML or in PDF format and saved in the current directory**

Before you get started, here are four small pieces of advice:

- to prepare your data visualisations, do not hesitate to create a separate, blank notebook; 


- pay particular attention to the readability and the clarity of your legends, titles, charts and tables; 


- the Pandas `resample` method might be useful if you want to plot time trends;


- keep in mind that you are working with real data and sometimes, data are not very talkative. Do not feel discouraged if your charts do not provide revolutionary insights: typically, an absence of trend is a substantial piece of information!

Good luck in changing Boston residents' lives!

# A word of conclusion

Congratulations for going through the exercise 🎉

If you wish to pursue your analysis at some point, note that all datasets (and many others) are publicly available online, on the [Analyze Boston](https://data.boston.gov) website.

Besides, if you are interested in the topic, you can start by reading the work of Nobel Prize laureate Gary Becker, who was the first to model crime as a rational phenomenon, similarly to an economic decision. This model, although it has limitations, marked a breakthrough in the study of crime and paved the way for many empirical studies that further analysed the socio-economic determinants of illegal activities. 

👉 [Link](https://olis.leg.state.or.us/liz/2017R1/Downloads/CommitteeMeetingDocument/125036) to download a full-text version of "Crime and Punishment: An Economic Approach" by Becker (1968)